# Endpoint Feature Test

### Create OpenAI client

In [35]:
from openai import OpenAI

client = OpenAI(
    api_key = "ocigenerativeai",
    base_url = "http://127.0.0.1:8088/v1/",
    )
models = client.models.list()

# Test chat completions
for model in models:
    print(model.id)


cohere.command-latest
cohere.command-a-03-2025
cohere.command-r-plus-08-2024
cohere.command-r-08-2024
meta.llama-latest
meta.llama-4-maverick-17b-128e-instruct-fp8
meta.llama-4-scout-17b-16e-instruct
meta.llama-3.3-70b-instruct
meta.llama-3.2-11b-vision-instruct
meta.llama-3.2-90b-vision-instruct
openai.gpt-4.1
openai.gpt-4.1-2025-04-14
openai.gpt-4.1-mini
openai.gpt-4.1-mini-2025-04-14
openai.gpt-4.1-nano
openai.gpt-4.1-nano-2025-04-14
openai.gpt-4o
openai.gpt-4o-2024-08-06
openai.gpt-4o-2024-11-20
openai.gpt-4o-mini
openai.gpt-4o-mini-2024-07-18
openai.gpt-4o-mini-search-preview
openai.gpt-4o-mini-search-preview-2025-03-11
openai.gpt-4o-search-preview
openai.gpt-4o-search-preview-2025-03-11
openai.o1
openai.o1-2024-12-17
openai.o3
openai.o3-2025-04-16
openai.o3-mini
openai.o3-mini-2025-01-31
openai.o4-mini
my-dedicated-model-name
my-datascience-model-name
ODSC-Mistral-7B-Instruct
ODSC-DeepSeek-R1-Distill-Qwen-7B
ODSC-sql-coder-graph


### Test chat completions with non-streaming response

In [39]:
test_models = [
    "openai.gpt-4o",
    "cohere.command-latest",
    "meta.llama-latest"
    ]

In [40]:
# Test chat completions with non-streaming response

for model_name in test_models:
    completion = client.chat.completions.create(
        model = model_name,
        messages = [{"role": "user", "content": "Hello!"}],
        max_tokens=12,
        )
    print(model_name, " : ", completion.choices[0].message.content)

openai.gpt-4o  :  Hello! How can I assist you today?
cohere.command-latest  :  Hello! How can I assist you today?
meta.llama-latest  :  Hello! It's nice to meet you. Is there something I


### Test chat completions with streaming response

In [41]:
for model in test_models:    
    print('\n',model," : ",end='')
    response = client.chat.completions.create(
        model=model,
        messages=[{'role': 'user', 'content': "hello"}],
        max_tokens=12,
        stream=True  # this time, we set stream=True
    )
    for chunk in response:        
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content,end='')


 openai.gpt-4o  : Hello! How can I assist you today?
 cohere.command-latest  : Hello! How can I assist you today?
 meta.llama-latest  : Hello! How are you today? Is there something I

### Test multi-modal with image input response

In [42]:
test_models = ["openai.gpt-4o"]

import base64
import mimetypes

image_path = "./test/image.jpg"
mime_type, _ = mimetypes.guess_type(image_path)
with open(image_path, "rb") as image_file:
    base64_str = base64.b64encode(image_file.read()).decode('utf-8')
url = f"data:{mime_type};base64,{base64_str}"
print(url[:100])

content = [
    {"type": "text","text": "describe this image?"},
	{"type": "image_url","image_url": {"url": url}}
    ]

for model in test_models:
    print('\n',"*"*20,"Model:",model,"*"*20)
    completion = client.chat.completions.create(
        model = model,
        messages = [{"role": "user", "content": content}],
        max_tokens=120,
        )
    print("Assistant:\n", completion.choices[0].message.content)

data:image/jpeg;base64,/9j/4AAQSkZJRgABAQEAeAB4AAD/4QAiRXhpZgAATU0AKgAAAAgAAQESAAMAAAABAAEAAAAAAAD/2

 ******************** Model: openai.gpt-4o ********************
Assistant:
 This image features a stylized, iconic logo in the shape of a red and yellow "S" shield. The background is blue, and the shield design is typically associated with a well-known superhero character.


### Test tool calls

In [52]:
def get_weather(latitude, longitude):
    import requests
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

#model = "cohere.command-a-03-2025"
#model = "meta.llama-3.3-70b-instruct"
model = "openai.gpt-4o"


#### Response without streaming

In [53]:
#### Cohere and Llama model without streaming

messages = [{"role": "user", "content": "Use tool to answer: What is the weather like in New York today?"}]

completion = client.chat.completions.create(
    model=model,
    messages=messages,
    tools=tools,
    stream=False
    )
print(f"Function call output for model -- {model}")
print(completion.choices[0].message.tool_calls,"\n")

new_message = completion.choices[0].message
tool_calls = completion.choices[0].message.tool_calls

Function call output for model -- openai.gpt-4o
[ChatCompletionMessageToolCall(id='call_x2VjsKQHoV8t7J5F6pZuSLBY', function=Function(arguments='{"latitude":40.7128,"longitude":-74.006}', name='get_weather'), type='function', index=0)] 



#### Response with streaming

In [51]:
#### Cohere and Llama model with streaming

messages = [{"role": "user", "content": "Use tool to answer: What is the weather like in New York today?"}]


response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        stream=True  # this time, we set stream=True
    )
print(model,":")
for chunk in response:    
    if not chunk.choices[0].delta.tool_calls:
        print(chunk.choices[0].delta.content,end='')
    else:
        if model.startswith("cohere"):
            print("\n")
        print(chunk.choices[0].delta.tool_calls)

openai.gpt-4o :
[ChoiceDeltaToolCall(index=0, id='call_BWZrMiww8u2hk22em2mdcCsv', function=ChoiceDeltaToolCallFunction(arguments='', name='get_weather'), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='{"', name=None), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='latitude', name=None), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='":', name=None), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='40', name=None), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='.', name=None), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='712', name=None), type='function')]
[ChoiceDeltaToolCall(index=0, id=None, function=ChoiceDeltaToolCallFunction(arguments='8',

In [54]:
#### Add messages to the conversation

messages.append(new_message)
for each in messages:
    print(each)

{'role': 'user', 'content': 'Use tool to answer: What is the weather like in New York today?'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_x2VjsKQHoV8t7J5F6pZuSLBY', function=Function(arguments='{"latitude":40.7128,"longitude":-74.006}', name='get_weather'), type='function', index=0)])


In [55]:
# use tool to get weather
import json
for tool_call in tool_calls:
    args = json.loads(tool_call.function.arguments)
    result = get_weather(args["latitude"], args["longitude"])
    print(result)
    # append result message
    messages.append({                               
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": str(result)
    })

for each in messages:
    print(each)

20.5
{'role': 'user', 'content': 'Use tool to answer: What is the weather like in New York today?'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_x2VjsKQHoV8t7J5F6pZuSLBY', function=Function(arguments='{"latitude":40.7128,"longitude":-74.006}', name='get_weather'), type='function', index=0)])
{'role': 'tool', 'tool_call_id': 'call_x2VjsKQHoV8t7J5F6pZuSLBY', 'content': '20.5'}


In [56]:
# Generate answer based on tool results without stream

completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=False
)
print(f"\nOutput for model -- {model}")
print(completion_2.choices[0].message.content)


Output for model -- openai.gpt-4o
The current temperature in New York is 20.5°C. Unfortunately, I don't have detailed information about other weather conditions at this time.


In [57]:
# Generate answer based on tool results with stream

completion_2 = client.chat.completions.create(
    model=model,
    messages=messages,
    stream=True
    )
print(f"\nOutput for model -- {model}")
for chunk in completion_2:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content,end='')


Output for model -- openai.gpt-4o
The weather in New York today is 20.5°C.

### Test embedding

In [ ]:
embd_model=[ 'cohere.embed-english-light-v3.0',
            'cohere.embed-english-v3.0',
            'cohere.embed-multilingual-light-v3.0',
            'cohere.embed-multilingual-v3.0']
input = ["hello!","你好！"]
for model in embd_model:
    print("model:",model)
    response = client.embeddings.create(input = input, model=model).data
    for each in response:
        print(each.index,':',str(each.embedding)[:36],'......',str(each.embedding)[-36:])
    print("*"*100)